In [1]:
from confluent_kafka import Consumer, KafkaError
from json import loads
from kafka import KafkaConsumer
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timedelta
from kafka import KafkaProducer

In [2]:
from tensorflow import keras
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import time
from tabulate import tabulate
from json import dumps

In [3]:
# Load trained LSTM model
model = keras.models.load_model('multistep_sbp_dbp_forecasting_model.h5')

In [4]:
producer = KafkaProducer(
    bootstrap_servers=['localhost:9093'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

In [5]:
# Kafka broker address
bootstrap_servers = 'localhost:9093'
 
# Kafka topic to consume messages from
kafka_topic = 'QM_data'
 
# Consumer group ID
group_id = 'group1'
 
# InfluxDB connection details
influxdb_host = 'localhost'
influxdb_port = 8086
influxdb_username = 'shams'
influxdb_password = 'shams123'
bucket = "BloodPressure"
organization = "OstProject"
influxdb_token = "FlRUvysHQ7GXF3N61Ii5VmtXn1xd-4-HYWxZRMSHmo8JJ_pODFGEkiPcBnYrET-blDswyQqheC0eRN1RA_ZJBA=="

# Create Kafka consumer configuration
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': group_id,
    'auto.offset.reset': 'earliest'
}

In [6]:
# Initialize InfluxDB client and write API
with InfluxDBClient(url="http://localhost:8086", token=influxdb_token, org=organization, bucket=bucket) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)

    # Your Kafka consumer setup...
    consumer = KafkaConsumer(kafka_topic, bootstrap_servers=['localhost:9093'], auto_offset_reset='earliest',
                             api_version=(0, 10), enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))

    # Initialize variables for rolling window
    time_steps = 10
    rolling_window_size = time_steps  # Each entry contains SBP and DBP
    rolling_window = []
    waiting_for_initial_10_points = True

    # Set the time interval between data consumption
    time_interval_seconds = 60

    # Initialize MinMaxScaler for input data scaling
    scaler = MinMaxScaler()

    try:
        for message in consumer:
            # Extract features based on your input data
            input_data = message.value

            sbp_value = float(input_data.get("SBP", ""))
            dbp_value = float(input_data.get("DBP", ""))

            # Skip input points where either SBP or DBP is 0
            if sbp_value == 0 or dbp_value == 0:
                print("")
                continue

            received_inputs = [
                sbp_value,
                dbp_value
            ]


            current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
            print(f"Received_input at {current_time}: {received_inputs}")

            # Send actual SBO and DBP values to InfluxDB
            actual_BP_values = Point("Actual_SBP_DBP").tag("your_tag_key", "your_tag_value") \
                    .field("SBP", sbp_value) \
                    .field("DBP", dbp_value)
             # Write data to InfluxDB
            write_api.write(bucket=bucket, org=organization, record=actual_BP_values)


            selected_features = [
                sbp_value,
                dbp_value
            ]

            # Append selected features to the rolling window
            rolling_window.append(selected_features)

            # Consume one data point every 1 minute
            time.sleep(time_interval_seconds)

            # If waiting for the initial 10 points
            if waiting_for_initial_10_points:
                # Check if the rolling window has accumulated 10 points
                if len(rolling_window) >= time_steps-1:
                    waiting_for_initial_10_points = False  # Set the flag to False after the initial 10 points
                    print("Initial 10 points accumulated, starting continuous forecasting.")

            # If not waiting for the initial 10 points
            else:
                # Check if the rolling window size is correct before reshaping
                if len(rolling_window) == time_steps:
                    # Convert the rolling window to a numpy array
                    rolling_window_array = np.array(rolling_window)

                    # Fit the scaler with the current rolling window
                    scaler.partial_fit(rolling_window_array)

                    # Scale the rolling window data
                    rolling_window_scaled = scaler.transform(rolling_window_array).reshape(1, time_steps, 2)

                    # Predict the next 30 minutes
                    future_predictions_scaled = model.predict(rolling_window_scaled)

                    # Inverse transform the predictions
                    future_predictions = scaler.inverse_transform(
                        np.array(future_predictions_scaled).reshape(-1, 2)).flatten()

                    # Convert the float32 values to regular Python floats
                    future_predictions = [float(value) for value in future_predictions]

                    # Separate SBP and DBP values
                    sbp_predictions = future_predictions[::2]
                    dbp_predictions = future_predictions[1::2]
                    timestamps = [datetime.utcnow() + timedelta(minutes=(i + 1)) for i in range(30)]


                    # Update the rolling window by removing the oldest data point
                    rolling_window = rolling_window[1:]

                    # Print the table
                    table_data = [
                        ["TIMESTAMP"] + [timestamp.strftime("%Y-%m-%d %H:%M") for timestamp in timestamps],
                        ["SBP"] + sbp_predictions,
                        ["DBP"] + dbp_predictions
                    ]

                    print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

                    # Send the 15th and 30th points to InfluxDB
                    sbp_15min = sbp_predictions[14]
                    dbp_15min = dbp_predictions[14]
                    sbp_30min = sbp_predictions[29]
                    dbp_30min = dbp_predictions[29]


                    # Send 15th point to InfluxDB
                    point_15min = Point("Forecast_15").tag("your_tag_key", "your_tag_value") \
                        .field("SBP", sbp_15min) \
                        .field("DBP", dbp_15min) 

                    # Send 30th point to InfluxDB
                    point_30min = Point("Forecast_30").tag("your_tag_key", "your_tag_value") \
                        .field("SBP", sbp_30min) \
                        .field("DBP", dbp_30min)

                    # Write data to InfluxDB
                    write_api.write(bucket=bucket, org=organization, record=point_15min)
                    write_api.write(bucket=bucket, org=organization, record=point_30min)

                    data_to_send = {
                    'sbp_predictions': sbp_predictions,
                    'dbp_predictions': dbp_predictions,
                    'timestamps': [timestamp.strftime("%Y-%m-%d %H:%M") for timestamp in timestamps]
                    }

                    # Produce the data to the new Kafka topic
                    producer.send('Forecasting_Results_Topic', value=data_to_send)

    except KeyboardInterrupt:
        pass


Received_input at 2023-12-19 01:16:26: [98.9, 63.1]
Received_input at 2023-12-19 01:17:26: [97.9, 63.0]
Received_input at 2023-12-19 01:18:26: [97.2, 62.5]
Received_input at 2023-12-19 01:19:26: [96.0, 62.0]
Received_input at 2023-12-19 01:20:26: [95.5, 61.7]
Received_input at 2023-12-19 01:21:26: [94.8, 61.4]
Received_input at 2023-12-19 01:22:26: [93.3, 60.5]
Received_input at 2023-12-19 01:23:26: [92.8, 60.4]
Received_input at 2023-12-19 01:24:26: [88.9, 57.9]
Initial 10 points accumulated, starting continuous forecasting.
Received_input at 2023-12-19 01:25:26: [93.2, 61.0]
1/1 [==============================] - 1s 733ms/step
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------